<a href="https://colab.research.google.com/github/Yuejun-Han/Capstone-Project/blob/main/2_Enformer_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction

This notebook demonstrates how to utlize Enformer to make gene expression prediction upon the data (from GEUVADIS/1000 genomes dataset) that we prepared from the previous notebook. 

### Major Steps

- Set up the environment
- Get/check the input data
- Make predictions and store them


## Setup

kipoiseq is a package that helps us to extract sequences from fasta files given some intervals. We will install the package. 

In [ ]:
!pip install kipoiseq==0.5.2 --quiet > /dev/null
# You can ignore the pyYAML error

Biopython is a python package that helps us do many bioinfomatic analysis in python

In [ ]:
!pip install Biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.3 MB 5.3 MB/s 


### Setting up our environments

Import packages that we need later. 

In [ ]:
import tensorflow_hub as hub # for interacting with saved models and tensorflow hub
import joblib
import gzip # for manipulating compressed files
import kipoiseq # for manipulating fasta files
from kipoiseq import Interval # same as above, really
import pyfaidx # to index our reference genome file
import pandas as pd # for manipulating dataframes
import numpy as np # for numerical computations
import pickle # for saving large objects
import os, sys # functions for interacting with the operating system
import random  

%matplotlib inline
%config InlineBackend.figure_format = 'retina'


connect to Google Drive to store results and data pernemently in cloud.   

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Because Enformer is a deep learning method that consumes high computational power, we are going to change the set up of Google Colab here to utlize GPU to help us speed up the prediction process. 

Google Colab gives us some GPU access. This limited GPU is available to anyone with a Google account, who has signed up to use Colaboratory. We will begin by changing the runtime type to GPU. Follow the instruction below by clicking on "Runtime -> Change runtime type -> GPU" in the menu bar below the title of this notebook. 

**Start the colab kernel with GPU**: Runtime -> Change runtime type -> GPU


Below, we import tensorflow as tf, and check that the runtime has been changed to GPU. 

In [ ]:
import tensorflow as tf
# Make sure the GPU is enabled 
assert tf.config.list_physical_devices('GPU'), 'Start the colab kernel with GPU: Runtime -> Change runtime type -> GPU'

Create a folder called "Capstone_Project_2022" 

In [ ]:
if not os.path.exists("/content/drive/MyDrive/Capstone_Project_2022/results/"):
  !mkdir -p "/content/drive/MyDrive/Capstone_Project_2022/results/"

In [ ]:
local_path = '/content/drive/MyDrive/Capstone_Project_2022/'

In [ ]:
transform_path = 'gs://dm-enformer/models/enformer.finetuned.SAD.robustscaler-PCA500-robustscaler.transform.pkl'
model_path = 'https://tfhub.dev/deepmind/enformer/1'
fasta_file = local_path+'genome.fa'

### Code

Next, we have some functions that will help us along the way. Classes and methods defined in this code block can be found in the [original Enformer usage colab notebook](https://colab.research.google.com/github/deepmind/deepmind_research/blob/master/enformer/enformer-usage.ipynb). 

In [ ]:
# @title `Enformer`, `EnformerScoreVariantsNormalized`, `EnformerScoreVariantsPCANormalized`,
SEQUENCE_LENGTH = 393216

class Enformer:

  def __init__(self, tfhub_url):
    self._model = hub.load(tfhub_url).model

  def predict_on_batch(self, inputs):
    predictions = self._model.predict_on_batch(inputs)
    return {k: v.numpy() for k, v in predictions.items()}

#  @tf.function
  def contribution_input_grad(self, input_sequence,
                              target_mask, output_head='human'):
    input_sequence = input_sequence[tf.newaxis]

    target_mask_mass = tf.reduce_sum(target_mask)
    with tf.GradientTape() as tape:
      tape.watch(input_sequence)
      prediction = tf.reduce_sum(
          target_mask[tf.newaxis] *
          self._model.predict_on_batch(input_sequence)[output_head]) / target_mask_mass

    input_grad = tape.gradient(prediction, input_sequence) * input_sequence
    input_grad = tf.squeeze(input_grad, axis=0)
    return tf.reduce_sum(input_grad, axis=-1)


class EnformerScoreVariantsRaw:

  def __init__(self, tfhub_url, organism='human'):
    self._model = Enformer(tfhub_url)
    self._organism = organism
  
  def predict_on_batch(self, inputs):
    ref_prediction = self._model.predict_on_batch(inputs['ref'])[self._organism]
    alt_prediction = self._model.predict_on_batch(inputs['alt'])[self._organism]

    return alt_prediction.mean(axis=1) - ref_prediction.mean(axis=1)


class EnformerScoreVariantsNormalized:

  def __init__(self, tfhub_url, transform_pkl_path,
               organism='human'):
    assert organism == 'human', 'Transforms only compatible with organism=human'
    self._model = EnformerScoreVariantsRaw(tfhub_url, organism)
    with tf.io.gfile.GFile(transform_pkl_path, 'rb') as f:
      transform_pipeline = joblib.load(f)
    self._transform = transform_pipeline.steps[0][1]  # StandardScaler.
    
  def predict_on_batch(self, inputs):
    scores = self._model.predict_on_batch(inputs)
    return self._transform.transform(scores)


class EnformerScoreVariantsPCANormalized:

  def __init__(self, tfhub_url, transform_pkl_path,
               organism='human', num_top_features=500):
    self._model = EnformerScoreVariantsRaw(tfhub_url, organism)
    with tf.io.gfile.GFile(transform_pkl_path, 'rb') as f:
      self._transform = joblib.load(f)
    self._num_top_features = num_top_features
    
  def predict_on_batch(self, inputs):
    scores = self._model.predict_on_batch(inputs)
    return self._transform.transform(scores)[:, :self._num_top_features]


# @title `variant_centered_sequences`

class FastaStringExtractor:
    
    def __init__(self, fasta_file):
        self.fasta = pyfaidx.Fasta(fasta_file)
        self._chromosome_sizes = {k: len(v) for k, v in self.fasta.items()}

    def extract(self, interval: Interval, **kwargs) -> str:
        # Truncate interval if it extends beyond the chromosome lengths.
        chromosome_length = self._chromosome_sizes[interval.chrom]
        trimmed_interval = Interval(interval.chrom,
                                    max(interval.start, 0),
                                    min(interval.end, chromosome_length),
                                    )
        # pyfaidx wants a 1-based interval
        sequence = str(self.fasta.get_seq(trimmed_interval.chrom,
                                          trimmed_interval.start + 1,
                                          trimmed_interval.stop).seq).upper()
        # Fill truncated values with N's.
        pad_upstream = 'N' * max(-interval.start, 0)
        pad_downstream = 'N' * max(interval.end - chromosome_length, 0)
        return pad_upstream + sequence + pad_downstream

    def close(self):
        return self.fasta.close()


def one_hot_encode(sequence):
  return kipoiseq.transforms.functional.one_hot_dna(sequence).astype(np.float32)

# @title `plot_tracks`

def plot_tracks(tracks, interval, height=1.5):
  fig, axes = plt.subplots(len(tracks), 1, figsize=(20, height * len(tracks)), sharex=True)
  for ax, (title, y) in zip(axes, tracks.items()):
    ax.fill_between(np.linspace(interval.start, interval.end, num=len(y)), y)
    ax.set_title(title)
    sns.despine(top=True, right=True, bottom=True)
  ax.set_xlabel(str(interval))
  plt.tight_layout()

Here, we define some utility functions for ourselves, to help us make predictions and analyse our predictions. 

In [ ]:
import Bio

from Bio.Seq import Seq
def create_rev_complement(dna_string):
    return(str(Seq(dna_string).reverse_complement()))

In [ ]:
def prepare_for_quantify_prediction_per_TSS(predictions, gene, tss_df):

  '''

  Parameters:
          predicitions (A numpy array): All predictions from the track
          gene (a gene name, character): a gene
          tss_df: a list of dataframe of genes and their transcription start sites
  Returns:
          A dictionary of cage experiment predictions and a list of transcription start sites

  '''
  
  output = dict()
  for tdf in tss_df:
    if gene not in tdf.genes.values:
      continue
    gene_tss_list = tdf[tdf.genes == gene].txStart_Sites.apply(str).values
    gene_tss_list = [t.split(', ') for t in gene_tss_list]
    gene_tss_list = [int(item) for nestedlist in gene_tss_list for item in nestedlist]
    gene_tss_list = list(set(gene_tss_list))
  output['cage_predictions'] = predictions[:, 5110] # a numpy array
  output['gene_TSS'] = gene_tss_list # a list


  return(output) # a dictionary

def quantify_prediction_per_TSS(low_range, TSS, cage_predictions):

  '''
  Parameters:
          low_range (int): The lower interval
          TSS (list of integers): A list of TSS for a gene
          cage_predictions: A 1D numpy array or a vector of predictions from enformer corresponding to track 5110 or CAGE predictions
  Returns:
          A dictionary of gene expression predictions for each TSS for a gene
    '''
  tss_predictions = dict()
  for tss in TSS:
    bin_start = low_range + ((768 + 320) * 128)
    count = -1
    while bin_start < tss:
      bin_start = bin_start + 128
      count += 1 
    if count >= len(cage_predictions)-1:
      continue
    cage_preds = cage_predictions[count - 1] + cage_predictions[count] + cage_predictions[count + 1]
    tss_predictions[tss] = cage_preds

  return(tss_predictions)

def collect_intervals(chromosomes, gene_list=None):

  '''
    Parameters : 
      chromosomes : a list of chromosome numbers; each element should be a string format
      gene_list : a list of genes; the genes should be located on those chromosomes

    Returns :
      A dictionary of genes (from gene_list) and their intervals within their respective chromosomes
  '''

  gene_intervals = {} # Collect intervals for our genes of interest

  for chrom in chromosomes:
    with open("/content/drive/MyDrive/Capstone_Project_2022/data/gene_chroms/gene_"+ chrom + ".txt", "r") as chrom_genes:
      for line in chrom_genes:
        split_line = line.strip().split("\t")
        gene_intervals[split_line[2]] = [
                                          split_line[0],
                                          int(split_line[3]),
                                          int(split_line[4])
                                        ]

  if isinstance(gene_list, list): # if the user has supplied a list of genes they are interested in
    use_genes = dict((k, gene_intervals[k]) for k in gene_list if k in gene_intervals)
    return(use_genes)
  elif isinstance(gene_list, type(None)):
    return(gene_intervals)


def run_predictions(gene_intervals, tss_dataframe, individuals_list=None):
  '''
  Parameters :
    gene_intervals : the results from calling `collect_intervals`
    tss_dataframe : a list of the TSSs dataframes i.e. the TSS for the genes in the chromosomes
    individuals_list : a list of individuals on which we want to make predictions; defaults to None

  Returns :
    A list of predictions; the first element is the predictions around the TSS for each gene. The second is the prediction across CAGE tracks
  '''

  gene_output = dict()
  gene_predictions = dict()

  for gene in gene_intervals.keys():
    gene_interval = gene_intervals[gene]
    target_interval = kipoiseq.Interval("chr" + gene_interval[0],
                                        gene_interval[1],
                                        gene_interval[2]) # creates an interval to select the right sequences
    target_fa = fasta_extractor.extract(target_interval.resize(SEQUENCE_LENGTH))  # extracts the fasta sequences, and resizes such that it is compatible with the sequence_length
    window_coords = target_interval.resize(SEQUENCE_LENGTH) # we also need information about the start and end locations after resizing
    try:
      cur_gene_vars = pd.read_csv("/content/drive/MyDrive/Capstone_Project_2022/data/individual_beds/chr" + gene_interval[0] + "/chr" + gene_interval[0] + "_"+ gene + ".bed", sep="\t", header=0) # read in the appropriate bed file for the gene
    except:
      continue
    individual_results = dict()
    individual_prediction = dict()

    if isinstance(individuals_list, list) or isinstance(individuals_list, type(np.empty([1, 1]))):
      use_individuals = individuals_list
    elif isinstance(individuals_list, type(None)):
      use_individuals = cur_gene_vars.columns[4:]

    for individual in use_individuals:
      print('Currently on gene {}, and predicting on individual {}...'.format(gene, individual))
      # two haplotypes per individual
      haplo_1 = list(target_fa[:])
      haplo_2 = list(target_fa[:])

      ref_mismatch_count = 0
      for i,row in cur_gene_vars.iterrows():

        geno = row[individual].split("|")
        if (row["POS"]-window_coords.start-1) >= len(haplo_2):
          continue
        if (row["POS"]-window_coords.start-1) < 0:
          continue
        if geno[0] == "1":
          haplo_1[row["POS"]-window_coords.start-1] = row["ALT"]
        if geno[1] == "1":
          haplo_2[row["POS"]-window_coords.start-1] = row["ALT"]

      # predict on the individual's two haplotypes
      prediction_1 = model.predict_on_batch(one_hot_encode("".join(haplo_1))[np.newaxis])['human'][0]
      prediction_2 = model.predict_on_batch(one_hot_encode("".join(haplo_2))[np.newaxis])['human'][0]

      temp_predictions = [prediction_1[:, 5110], prediction_2[:, 5110]] # CAGE predictions we are interested in 
      individual_prediction[individual] = temp_predictions

      # Calculate TSS CAGE expression which correspond to column 5110 of the predictions above
      temp_list = list()

      pred_prepared_1 = prepare_for_quantify_prediction_per_TSS(predictions=prediction_1, gene=gene, tss_df=tss_dataframe)
      tss_predictions_1 = quantify_prediction_per_TSS(low_range = window_coords.start, TSS=pred_prepared_1['gene_TSS'], cage_predictions=pred_prepared_1['cage_predictions'])

      pred_prepared_2 = prepare_for_quantify_prediction_per_TSS(predictions=prediction_2, gene=gene, tss_df=tss_dataframe)
      tss_predictions_2 = quantify_prediction_per_TSS(low_range = window_coords.start, TSS=pred_prepared_2['gene_TSS'], cage_predictions=pred_prepared_2['cage_predictions'])

      temp_list.append(tss_predictions_1)
      temp_list.append(tss_predictions_2) # results here are a dictionary for each TSS for each haplotype

      individual_results[individual] = temp_list # save for the individual

    gene_output[gene] = individual_results
    gene_predictions[gene] = individual_prediction

  return([gene_output, gene_predictions])


def collect_target_intervals(gene_intervals):

  '''
  Returns a dictionary of Interval objects (from kipoiseq) for each gene corresponding to the locations of the gene
  '''

  target_intervals_dict = dict()

  for gene in gene_intervals.keys():
    gene_interval = gene_intervals[gene]
    target_interval = kipoiseq.Interval("chr" + gene_interval[0],
                                        gene_interval[1],
                                        gene_interval[2])
    target_intervals_dict[gene] = target_interval

  return(target_intervals_dict)

In [ ]:
def plot_predixcan_vs_geuvadis(interested_gene, interested_individuals, geuvadis_expression, predixcan_expression):

  '''
  Show a plot and return correlation coefficient
  '''
  # from predixcan expression
  df_predixcan = predixcan_expression[predixcan_expression.gene_name == interested_gene].loc[:,interested_individuals]
  # from enformer
  df_geuvadis = geuvadis_expression[geuvadis_expression.gene_name == interested_gene].loc[:,interested_individuals]

  # concatenate both
  df_all = pd.concat([df_predixcan, df_geuvadis], axis=0)
  df_all.index = ['Predixcan', 'GEUVADIS']

  # plotting
  sns.regplot(x=df_all.iloc[0,:], y=df_all.iloc[1,:], color='red').set(title='Predixcan vs. GEUVADIS predictions on {} individuals for gene {}'.format(len(df_all.columns), interested_gene))

  # correlation coefficient
  corr_coef = np.corrcoef(x=df_all.iloc[0,:], y=df_all.iloc[1,:])[0][1]

  return([df_all, corr_coef])

def plot_enformer_vs_predixcan(prediction_results, interested_gene, interested_individuals, predixcan_expression, how='sum'):

  '''
  Show a plot and return correlation coefficient
  '''

  enformer_predictions = dict()

  for gene, individuals in prediction_results[0].items():
    temp_individual = dict()
    for individual, haplo_predictions in individuals.items():
      temp = list()
      for i in range(0, len(haplo_predictions[0])):
        temp.append(list(haplo_predictions[0].values())[i] + list(haplo_predictions[1].values())[i])
      if how == 'sum':
        temp_individual[individual] = np.sum(temp)
      elif how == 'max':
        temp_individual[individual] = np.max(temp)
    enformer_predictions[gene] = temp_individual

  # from predixcan expression
  df_predixcan = predixcan_expression[predixcan_expression.gene_name == interested_gene].loc[:,interested_individuals]
  # from enformer
  df_enformer = pd.DataFrame(enformer_predictions[interested_gene], index=[0]).loc[:, df_predixcan.columns]

  # concatenate both
  df_all = pd.concat([df_enformer, df_predixcan], axis=0)
  df_all.index = ['Enformer', 'Predixcan']

  # plotting
  sns.regplot(x=df_all.iloc[0,:], y=df_all.iloc[1,:], color='red').set(title='Predixcan vs. Enformer predictions on {} individuals for gene {}'.format(len(df_all.columns), interested_gene))

  # correlation coefficient
  corr_coef_predix = np.corrcoef(x=df_all.iloc[0,:], y=df_all.iloc[1,:])[0][1]

  return([df_all, corr_coef_predix])


def plot_enformer_vs_geuvadis(prediction_results, interested_gene, interested_individuals, geuvadis_expression, how='sum'):

  '''
  Show a plot and return correlation coefficient
  '''

  enformer_predictions = dict()

  for gene, individuals in prediction_results[0].items():
    temp_individual = dict()
    for individual, haplo_predictions in individuals.items():
      temp = list()
      for i in range(0, len(haplo_predictions[0])):
        temp.append(list(haplo_predictions[0].values())[i] + list(haplo_predictions[1].values())[i])
      if how == 'sum':
        temp_individual[individual] = np.sum(temp)
      elif how == 'max':
        temp_individual[individual] = np.max(temp)
    enformer_predictions[gene] = temp_individual

  # from geuvadis expression
  df_geuvadis = geuvadis_expression[geuvadis_expression.gene_name == interested_gene].loc[:,interested_individuals]
  #df_enformer = np.transpose(pd.DataFrame(enformer_predictions)).loc[:, df_geuvadis.columns]
  df_enformer = pd.DataFrame(enformer_predictions[interested_gene], index=[0]).loc[:, df_geuvadis.columns]

  # concatenate both
  df_all = pd.concat([df_enformer, df_geuvadis], axis=0)
  df_all.index = ['Enformer', 'GEUVADIS']

  # plotting
  sns.regplot(x=df_all.iloc[0,:], y=df_all.iloc[1,:], color='blue').set(title='Enformer vs. Geuvadis predictions on {} individuals for gene {}'.format(len(df_all.columns), interested_gene))
  
  # correlation coefficient
  corr_coef_geu = np.corrcoef(x=df_all.iloc[0,:], y=df_all.iloc[1,:])[0][1]

  return([df_all, corr_coef_geu])


# Run Prediction

To make predictions we use one of the functions defined in the **'Code'** section, namely : *run_predictions*. It requires three parameters: gene_intervals, tss_dataframe, and an individuals_list. Let's prepare them, and we can check whether they are all avaliable in our drive along the way.

## Prepare Gene Interval

1. set index as the gene name
2. transpose the dataframe so the column name is now the gene name, and the column values are chr, start, and end.
3. because the function: to_dict, 'list' - keys are column names, values are lists of column data

In [ ]:
path = local_path + 'data/target_gene_info.csv'
df_full_target_genes = pd.read_csv(path, index_col = 'Unnamed: 0')

In [ ]:
df_full_target_genes['chr'] = df_full_target_genes['chr'].astype(str)

In [ ]:
df_full_target_genes.set_index('gene_name').T

gene_name,SERPINB1,KBTBD2,GCDH,ZNF493,MANBA,ZNF586,THOC3,ZNF567,PCYT1A,PAPD7,...,HELB,SLC16A9,FCRL3,MCM8,MRPL21,FAAH,SLC36A1,PFN2,COG3,ICAM5
chr,6,7,19,19,4,19,5,19,3,5,...,12,10,1,20,11,1,5,3,13,19
start,2832566,32907784,13001974,21579931,103552660,58280997,175344876,37178530,195941093,6714718,...,66696325,61410523,157646271,5931298,68658744,46859937,150816607,149682691,46039060,10400655
end,2842240,32933743,13010783,21610375,103682151,58320617,175461683,37214248,196014828,6757161,...,66737423,61495760,157670775,5975852,68671303,46879520,150871942,149768575,46110765,10407453


In [ ]:
my_intervals = df_full_target_genes.set_index('gene_name').T.to_dict('list')

Now we have a list (*my_intervals*) containing 100 genes of our interest for one of the parameters of the function.

In [ ]:
list_of_genes = df_full_target_genes['gene_name'].to_list()  # prepare a list of genes for plotting

## TSS Dataframe Preparation

Read into a list, the list contains dataframes that are the TSS per gene for the chromosomes we are interested in 

In [ ]:
# a list containing all the chromosome numbers that we have resource for
chromosomes = ['6','7','19','4','5','3','18','21','1','15','22','11','2','17','12','9','16','14','10','13','20']

In [ ]:
tss = []
for chr in chromosomes:
  df = pd.read_table('/content/drive/MyDrive/Capstone_Project_2022/data/tss_by_chr/chr' +chr+ '_tss_by_gene.txt', sep='\t')
  tss.append(df)

## List of Individuals Preparation

In [ ]:
path1 = "/content/drive/MyDrive/Capstone_Project_2022/data/CEU_sample_group.csv"
CEU_sample_group = pd.read_csv(path1)

In [ ]:
path2 = "/content/drive/MyDrive/Capstone_Project_2022/data/YRI_sample_group.csv"
YRI_sample_group = pd.read_csv(path2)

## Run Predictions

We have everything ready now, we can start generating haplotype sum predictions for YRI and CEU sample groups. This would take a while.

We still need the model itself. The model has been graciously hosted on [Tensorflow Hub](https://tfhub.dev/deepmind/enformer/1), which hosts many other models too. You can click on the link and explore. When you click the link, you can see that the model is about 892 Mb large. Quite big. 
We will use the url to the model to download and use it here. 

Earlier, we defined an Enformer class (see the codes section). We will load the model into this class. The model has been trained and the weights are freely available. All we need to do is to load this model and use it. Neat. 

We also defined a class FastaStringExtractor, that can help us extract raw sequences from fasta files given the intervals we want. We will make use of this class too.

In [ ]:
model = Enformer(model_path) # here we load the model architecture. 

fasta_extractor = FastaStringExtractor(fasta_file) # we define a class called fasta_extractor to help us extra raw sequence data

for YRI group


In [ ]:
import timeit
test = YRI_sample_group["individual"].values.tolist()
filepath = '/content/drive/MyDrive/Capstone_Project_2022/results/YRI_Haplotype_Sum.csv'

# Making sure the file exists before writing into it. 
if not os.path.exists(filepath):
  df_YRI_Hap_Sum = pd.DataFrame(index = test, columns = list_of_genes)
  df_YRI_Hap_Sum.to_csv(filepath)
  
list_of_genes = df_full_target_genes['gene_name'].to_list()  # prepare a list of genes for plotting

start = timeit.default_timer()

# i is index of the individual list
for i in range(len(test)): 
  df_YRI_Hap_Sum = pd.read_csv(filepath, index_col = 'Unnamed: 0', na_values='NaN', keep_default_na=False)
  if df_YRI_Hap_Sum.at[test[i],list_of_genes[0]]: 
    #print('value exists')
    continue
  else:
    test_predictions = run_predictions(gene_intervals=my_intervals, tss_dataframe=tss, individuals_list=test[i:i+1]) # here we make predictions and save it.
    for gene in range(len(list_of_genes)):
      df_YRI_Hap_Sum.at[test[i],list_of_genes[gene]]=pd.DataFrame(test_predictions[0][list_of_genes[gene]][test[i]]).sum().sum()
    df_YRI_Hap_Sum.to_csv(filepath)
    
stop = timeit.default_timer()
print('Time: ', stop - start)

for CEU group


In [ ]:
test = CEU_sample_group["individual"].values.tolist()
filepath = '/content/drive/MyDrive/Capstone_Project_2022/results/CEU_Haplotype_Sum.csv'

# Check whether there is a file named CEU_Haplotype_Sum.csv, if not create one
if not os.path.exists(filepath):
  df_CEU_Haplotype_Sum = pd.DataFrame(index = test, columns = list_of_genes)
  df_CEU_Haplotype_Sum.to_csv(filepath)

list_of_genes = df_full_target_genes['gene_name'].to_list()  # prepare a list of genes for plotting

start = timeit.default_timer()

# i is index of the individual list
for i in range(len(test)): 
  df_hapScoreSum = pd.read_csv(filepath, index_col = 'Unnamed: 0', na_values='NaN', keep_default_na=False)
  if df_hapScoreSum.at[test[i],list_of_genes[0]]: 
    #print('value exists')
    continue
  else:
    test_predictions = run_predictions(gene_intervals=my_intervals, tss_dataframe=tss, individuals_list=test[i:i+1]) # here we make predictions and save it.
    for gene in range(len(list_of_genes)):
      df_hapScoreSum.at[test[i],list_of_genes[gene]]=pd.DataFrame(test_predictions[0][list_of_genes[gene]][test[i]]).sum().sum()
    df_hapScoreSum.to_csv(filepath)

stop = timeit.default_timer()
print('Time: ', stop - start)

**By now there should be two csv files in your drive. We can move on to the next script where we will plot some graphs for visulization and do some analysis.**  